# Import library

In [1]:
import os
import yaml
import openai
from openai import OpenAI

with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

# LangChain Expression Language (LCEL)

## Simple example

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke(
    {'topic': "bears"}
)

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

## Retrieval example

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [8]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [9]:
retriever.get_relevant_documents("what do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

# Runnable map

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap

In [12]:
chain = RunnableMap({
    'context': lambda x: retriever.get_relevant_documents(x['question']),  # call retriever
    'question': lambda x: x['question']
}) | prompt | model | output_parser

In [13]:
chain.invoke({'question': "where did harrison work?"})

'Harrison worked at Kensho.'

In [14]:
inputs = RunnableMap({
    'context': lambda x: retriever.get_relevant_documents(x['question']),  # call retriever
    'question': lambda x: x['question']
})

In [15]:
inputs.invoke({'question': "where did harrison work?"})  # dict

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

# Bind

In [16]:
functions = [
    {
        'name': 'weather_search',
        'description': 'Search for weather given an airport code',
        'parameters': {
            'type': 'object',
            'properties': {
                'airport_code': {
                    'type': 'string',
                    'description': "The airport code to get the weather for"
                }
            },
            'required': ['airport_code']
        }
    }
]

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('human', '{input}')
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [18]:
runnable = prompt | model

In [19]:
runnable.invoke({'input': "What is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}})

In [20]:
functions = [
    {
        'name': 'weather_search',
        'description': 'Search for weather given an airport code',
        'parameters': {
            'type': 'object',
            'properties': {
                'airport_code': {
                    'type': 'string',
                    'description': "The airport code to get the weather for"
                }
            },
            'required': ['airport_code']
        }
    },

    {
        'name': 'sports_search',
        'description': 'Search for news of recent sport events',
        'parameters': {
            'type': 'object',
            'properties': {
                'team_name': {
                    'type': 'string',
                    'description': "The sports team to search for"
                }
            },
            'required': ['tema_name']
        }
    }
]

In [21]:
model = model.bind(functions=functions)

In [22]:
runnable = prompt | model

In [23]:
runnable.invoke({'input': "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}})

# Fallbacks

- `langchain.llm.OpenAI`: old model not good at outputting JSON

In [24]:
from langchain.llms import OpenAI
import json

In [25]:
simple_model = OpenAI(
    temperature=0,  # determinism
    max_tokens=1000,
    model='gpt-3.5-turbo-instruct'
)
simple_chain = simple_model | json.loads

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [26]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

- `JSONDecodeError` 발생

In [27]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [28]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [29]:
final_chain = simple_chain.with_fallbacks([chain])  # other runnables
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': "Softly it blows, the wind's gentle touch"},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In moonlit night, a song unheard'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows sway, a graceful ballet'}}

# Interface

In [30]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [31]:
chain.invoke({'topic': "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

## Execute batch processing **in parallel as much as possible**

In [32]:
chain.batch([
    {'topic': "bears"},
    {'topic': "frogs"}
])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 "Why don't frogs like to play basketball?\nBecause they always get too hoppy!"]

## Streaming output

In [33]:
for t in chain.stream({'topic': "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



## Asynchronous methods

In [34]:
response = await chain.ainvoke({'topic': "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"